In [ ]:
#Importa librerias necesarias
import time
import pandas as pd
from selenium import webdriver
from pymongo import MongoClient

In [ ]:
#Asigna la url de tu portal empleo 
url = "https://www.tuportalempleo.com/buscar-ofertas-de-empleo-actuales"

In [ ]:
#Conecta con mongodb
Client = MongoClient('localhost',27017)

In [ ]:
#Lista las bases de mongodb
Client.list_database_names()

In [ ]:
#Selecciona la base alldata
db = Client.alldata
db

In [ ]:
#Lista las colecciones
colecciones = db.list_collection_names()
colecciones

In [ ]:
#Selecciona la coleccion creada para tuportalempleo
collection = db["tuportalempleo"]
collection

In [1]:
#Crea el metodo
def extraer_ofertas(driver):
    print('Realizando scraping...')
    i=1
    try:
        #guarda las ofertas en una lista
        ofertas = driver.find_elements_by_xpath('//*[@id="tablaOfertas"]/div[@class="Ofertas"]/div[@class="oferta"]')
        #recorre la lista
        while i <= len(ofertas):
            #vuelve a guardar la lista de las ofertas para actualizarla
            ofertas = driver.find_elements_by_xpath('//*[@id="tablaOfertas"]/div[@class="Ofertas"]/div[@class="oferta"]')
            #selecciona la oferta deacuerdo a la posicion del while
            oferta=ofertas[i]
            #convierte la posicion en string
            a=str(i)
            #selecciona la oferta segun el string y le da click
            oferta.find_element_by_xpath('//*[@id="tablaOfertas"]/div/div['+a+']/a/div[1]/h3').click()
            #agrega un tiempo de demora
            time.sleep(5)
            #obtiene los datos de la oferta
            cargo = driver.find_element_by_xpath('//*[@id="tituloOferta"]/h1').text
            tipocontrato = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[2]/div/input').get_attribute('value')
            ciudad = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div/input').get_attribute('value')
            fechapublicacion = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[2]/div/input').get_attribute('value')
            remuneracion = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[3]/div/input').get_attribute('value')
            categoria = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div/input').get_attribute('value')
            sectorlaboral = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[4]/div/input').get_attribute('value')
            contrato = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[3]/div/input').get_attribute('value')
            contenido = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[1]/div[2]/div[2]/div[1]/div/div').text.replace('\n',"")
            
            #guarda en un diccionario
            oferta_dict = {
                'portalweb':"tuportalempleo",
                'cargo':cargo,
                'tipocontrato':tipocontrato,
                'ciudad':ciudad,
                'fechapublicacion':fechapublicacion,
                'remuneracion':remuneracion,
                'categoria':categoria,
                'sectorlaboral':sectorlaboral,
                'contrato':contrato,
                'contenido':contenido
            }
            
            #consulta si el empleo ya existe
            if collection.find_one(oferta_dict):
                #si existe regresa
                driver.back()
            else:
                #si no existe inserta en la coleccion
                collection.insert_one(oferta_dict)
                #regresa
                driver.back()
            #agrega un tiempo de demora
            time.sleep(5)
            i+= 1
    except:
        print('Terminado')
        

In [ ]:
#Inicia el web driver desde su ubicacion en el equipo
driver = webdriver.Chrome(executable_path='C:/Users/DELL/Downloads/chromedriver.exe')

In [ ]:
#Ingresa a la url
driver.get(url)

In [ ]:
#Ejecuta el metodo
extraer_ofertas(driver)